<a href="https://colab.research.google.com/github/srohit0/CORDIC/blob/master/ABCs_Of_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [0]:
# Tensors
x = torch.FloatTensor([[1,2,3], [4,5,6]])
print(x.size(), "\n", x)

In [0]:
# Add tensors
x.add_(torch.ones([2,3])+torch.ones([2,3]))

In [0]:
# Subtract Tensor
x.sub_(torch.ones([2,3])*2)

In [0]:
# Numpy to torch tensors
import numpy as np

y = np.matrix([[2,2],[2,2],[2,2]])
z = np.matrix([[2,2],[2,2],[2,2]], dtype='int16')
x.short() @ torch.from_numpy(z)

In [0]:
# Reshape tensors (similar to np.reshape)
x.view(1,6)

In [0]:
# move variables and copies across compute devices.

x = torch.FloatTensor([[1,2,3], [4,5,6]])
y = np.matrix([[2,2,2],[2,2,2]], dtype='float32')

if ( torch.cuda.is_available() ):
  x = x.cuda();
  y = torch.from_numpy(y).cuda()
  z = x+y
print(z)

print(x.cpu())

In [0]:
# Variable (part of autograd package)
# Variables (graph nodes) are thin wrappers around tensors and have dependency knowledge
# Variables enable backpropagation of gradients and automatic differentiations
# Variable are set a 'volatile' flad during infrencing.

from torch.autograd import Variable
v1 = Variable(torch.tensor([1.,2.,3.]), requires_grad=False)
v2 = Variable(torch.tensor([4.,5.,6.]), requires_grad=True)
v3 = v1 * v2 

v3.data.numpy()

In [0]:
# Variables remember what created them.
v3.grad_fn

In [0]:
# Backpropagation with example of sin(x)
x = Variable(torch.Tensor(np.array([0., 1., 1.5, 2.])*np.pi), requires_grad=True)
y = torch.sin(x)
x.grad
y.backward(torch.Tensor([1.,1.,1.,1]))

# Check gradient is indeed cox(x)
if ( (x.grad.data.int().numpy() == torch.cos(x).data.int().numpy()).all() ):
  print ("d(sin(x)/dx = cos(x))")

In [0]:
# Simple Liner Regression
#         Fit a line to the data. Y = w.x + b

# Deterministic behavior
np.random.seed(0)
torch.manual_seed(0)


# Step 1: Dataset

w = 2; b = 3
x = np.linspace(0, 10, 100)
y = w*x + b + np.random.randn(100)*2
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

In [0]:
# Step 2: Model

class LinearRegressionModel(torch.nn.Module):
  
  def __init__(self, in_dimn, out_dimn):
    super(LinearRegressionModel, self).__init__()
    self.model = torch.nn.Linear(in_dimn, out_dimn)
    
  def forward(self, x):
    y_pred = self.model(x);
    return y_pred;

model = LinearRegressionModel(in_dimn=1, out_dimn=1)

In [0]:
# Step 3: Training

cost      = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
inputs    = Variable(torch.from_numpy(x.astype('float32')))
outputs   = Variable(torch.from_numpy(y.astype('float32')))

for epoch in range(100):
  # 3.1 forward pass:
  y_pred = model(inputs)
  
  # 3.2 compute loss
  loss = cost(y_pred, outputs)
  
  # 3.3 backward pass
  optimizer.zero_grad(); # by default, gradient accumulate
  loss.backward()
  optimizer.step()
  if ( (epoch+1) % 10 == 0 ):
    print('epoch {}, loss {}'.format(epoch+1, loss.data)) 

In [0]:
# Step 4: Display model and confirm

import matplotlib.pyplot as plt
plt.figure(figsize=(4,4))
plt.title('Model and Dataset')
plt.xlabel('X'); plt.ylabel('Y')
plt.grid()
plt.plot(x, y, 'ro', label='Dataset', marker='x', markersize=4)
plt.plot(x, model.model.weight.item()*x+model.model.bias.item(), label='Regression Model')
plt.legend(); plt.show()